## Importa funções para o pré-processamento

In [13]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd
import numpy as np

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carregamento dos dados

Para a realizar a busca por arquivos com extensão .dlis na pasta Dados-dlis, foi utilizada a função **glob()** da biblioteca glob.

A leitura dos dados é feita com a função **load()** da biblioteca dlisio.

São armazenados três tipos de dados:
* **Nome do arquivo** na lista **nomes_arquivos**

* **Dados das curvas** na lista **leituras_dlis**

* **Nome do poço** na lista **nomes_anp**

In [14]:
nomes_arquivos = []     # Armazena os nomes dos arquivos .dlis
leituras_dlis = []      # Armazena as leituras dos arquivos .dlis
nomes_anp = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Dados-dlis' + "/*.dlis", recursive=True):
    try:
        # Salva o nome do arquivo
        nomes_arquivos.append(file)

        # Salva os dados da leitura
        leitura, *tail = dlis.load(f'{file}')
        leituras_dlis.append(leitura)

        # Salva o nome do poço
        nome = leitura.origins[0].well_name
        nomes_anp.append(nome)
    except:
        pass

## Cria dicionário para armazenar os dados e respectivos nomes

In [17]:
# Casa itens da lista 'nome_anp_abreviados' com os itens da lista 'leituras_dlis'
pares = zip(nomes_anp, leituras_dlis)

# Cria dicionário 'dli_dict'
dli_dict = dict(pares)
dli_dict

{'1-FSG-1-SE': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '1-FSJQ-1-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-595-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-605-SE': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '1-BRSA-659-SE': LogicalFile(GEOLOAD.1),
 '1-POI-1-SE': LogicalFile(AIT_SONIC_TLD_MCFL_024PUP),
 '1-BRSA-696-SE': LogicalFile(AIT_SONIC_TLD_MCFL_003PUP),
 '1-DP-2-SE': LogicalFile(AIT_SONIC_TLD_MCFL_079PUP),
 '1CSC 0001  SE': LogicalFile(1CSC 0001  SEBRA1CSC 0001  SE1),
 '1FBV 0001  SE': LogicalFile(1FBV 0001  SEBRA1FBV 0001  SE1),
 '1PRP 0001  SE': LogicalFile(1PRP 0001  SEBRA1PRP 0001  SE1)}

## Separa TODAS as curvas presentes nos .dlis de cada poço
#### Para visualizar as curvas obtidas a partir da extração dos .dlis serão criados arquivos .csv com essas curvas presentes nos .dlis de cada poço.

In [18]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels_list = []
    for frame in poco.frames:
        channels = frame.channels
        channels_list.append([channel.name for channel in channels])

    channels_dict[key] = sum(channels_list, [])

In [19]:
import csv

# Para cada poço no dicionário channels_dict, criar um arquivo CSV
for key, channels in channels_dict.items():
    # Definir o nome do arquivo CSV usando o identificador do poço
    filename = f'Curvas_CSV/{key}.csv'
    
    # Abrir o arquivo em modo de escrita
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Escrever o cabeçalho do CSV
        writer.writerow(['Poco', 'Channel'])
        
        # Escrever uma linha para cada canal, incluindo o nome do poço
        for channel in channels:
            writer.writerow([key, channel])

print("Arquivos CSV criados com sucesso!")

Arquivos CSV criados com sucesso!


## Concatena os arquivos CSV das curvas em um único arquivo
#### Para visualizar todas as curvas da extração dos .dlis em um único arquivo, os .csv de cada poço serão concatenados em um único arquivo .csv.

In [20]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=0, ignore_index=True)

# Salva o DF concatenado em um .csv
df_concat.to_csv("Curvas_CSV/curvas_pocos.csv", index=False)

print("Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso")


Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso


## Curvas escolhidas

In [21]:
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'HDRA', 'BSZ', 'BS', 'HCAL', 'CAL', 'CALI', 'DCALI', 'DCAL', 'PE', 'PEFZ', 'PEU', 'DT', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']

## Processa os 'frames' dos poços 1-BRSA-551-SE e 1-BRSA-605-SE

#### Quantidade de 'frames' dos poços

In [22]:
poco_551 = dli_dict['1-FSG-1-SE']
poco_605 = dli_dict['1-BRSA-605-SE']
poco_1_POI_1_SE = dli_dict['1-POI-1-SE']
poco_696 = dli_dict['1-BRSA-696-SE']
poco_1_DP_2_SE = dli_dict['1-DP-2-SE']
poco_1CSC_0001__SE = dli_dict['1CSC 0001  SE']
poco_1FBV_0001__SE = dli_dict['1FBV 0001  SE']
poco_1PRP_0001__SE = dli_dict['1PRP 0001  SE']

print(poco_551.frames)
print(poco_605.frames)
print(poco_1_POI_1_SE.frames)
print(poco_696.frames)
print(poco_1_DP_2_SE.frames)
print(poco_1CSC_0001__SE.frames)
print(poco_1FBV_0001__SE.frames)
print(poco_1PRP_0001__SE.frames)

[Frame(60B), Frame(10B), Frame(5B), Frame(30B), Frame(120B), Frame(180B), Frame(20B), Frame(1B), Frame(2B)]
[Frame(60B), Frame(10B), Frame(30B), Frame(20B)]
[Frame(60B), Frame(10B), Frame(5B), Frame(30B), Frame(120B), Frame(180B), Frame(20B), Frame(1B), Frame(2B)]
[Frame(60B), Frame(10B), Frame(5B), Frame(30B), Frame(120B), Frame(180B), Frame(20B), Frame(1B), Frame(2B)]
[Frame(60B), Frame(10B), Frame(5B), Frame(30B), Frame(120B), Frame(180B), Frame(20B), Frame(1B), Frame(2B)]
[Frame([0,1,IEL/BCS/GR/BOREHOLE-DEPTH/1/.20/M])]
[Frame([0,1,IEL/BCS/GR/BOREHOLE-DEPTH/1/1.0/M])]
[Frame([0,1,COMCEN.001/BOREHOLE-DEPTH/1/.005085110664368/M])]


In [23]:
print(dli_dict['1-BRSA-659-SE'].frames[0].attic['SPACING'].units)
print(poco_551.frames[0].attic['SPACING'].units)

m
0.1 in


#### Cria um dicionário para armazenar os 'frames' 

In [24]:
frames_dict_551 = cria_frames_dict(poco_551)
frames_dict_605 = cria_frames_dict(poco_605)
frames_dict_1_POI_1_SE = cria_frames_dict(poco_1_POI_1_SE)
frames_dict_696 = cria_frames_dict(poco_696)
frames_dict_1_DP_2_SE = cria_frames_dict(poco_1_DP_2_SE)
frames_dict_1CSC_0001__SE = cria_frames_dict(poco_1CSC_0001__SE)

#### Cria um 'dataframe' para cada 'frame'

In [25]:
dataframes_dict_551 = {}
dataframes_dict_605 = {}
dataframes_dict_1_POI_1_SE = {}
dataframes_dict_696 = {}
dataframes_dict_1_DP_2_SE = {}
dataframes_dict_1CSC_0001__SE = {}

cria_dataframes_dict(frames_dict_551, dataframes_dict_551, curvas_escolhidas)
cria_dataframes_dict(frames_dict_605, dataframes_dict_605, curvas_escolhidas)
cria_dataframes_dict(frames_dict_1_POI_1_SE, dataframes_dict_1_POI_1_SE, curvas_escolhidas)
cria_dataframes_dict(frames_dict_696, dataframes_dict_696, curvas_escolhidas)
cria_dataframes_dict(frames_dict_1_DP_2_SE, dataframes_dict_1_DP_2_SE, curvas_escolhidas)
cria_dataframes_dict(frames_dict_1CSC_0001__SE, dataframes_dict_1CSC_0001__SE, curvas_escolhidas)

In [26]:
dataframes_dict_1CSC_0001__SE[0]

,CALI,DT,GR,ILD
0,-999.25,-999.25,33.193314,-999.250000
1,-999.25,-999.25,36.426758,-999.250000
2,-999.25,-999.25,39.074066,-999.250000
3,-999.25,-999.25,39.733582,-999.250000
4,-999.25,-999.25,41.318863,-999.250000
...,...,...,...,...
6830,-999.25,-999.25,-999.250000,1.795197
6831,-999.25,-999.25,-999.250000,1.802460
6832,-999.25,-999.25,-999.250000,1.807922
6833,-999.25,-999.25,-999.250000,1.811338


#### Move a vírgula uma casa para a esquerda em todos os valores das colunas TDEP

In [27]:
for value in dataframes_dict_551.values():
    value["TDEP"] = value["TDEP"] / 10

for value in dataframes_dict_605.values():
    value["TDEP"] = value["TDEP"] / 10

for value in dataframes_dict_1_POI_1_SE.values():
    value["TDEP"] = value["TDEP"] / 10

for value in dataframes_dict_696.values():
    value["TDEP"] = value["TDEP"] / 10

for value in dataframes_dict_1_DP_2_SE.values():
    value["TDEP"] = value["TDEP"] / 10

#### Converte de polegada para metro

In [28]:
for value in dataframes_dict_551.values():
    value["TDEP"] = value["TDEP"] * 0.0254

for value in dataframes_dict_605.values():
    value["TDEP"] = value["TDEP"] * 0.0254
    
for value in dataframes_dict_1_POI_1_SE.values():
    value["TDEP"] = value["TDEP"] * 0.0254

for value in dataframes_dict_696.values():
    value["TDEP"] = value["TDEP"] * 0.0254

for value in dataframes_dict_1_DP_2_SE.values():
    value["TDEP"] = value["TDEP"] * 0.0254    

#### Salva os dataframes como arquivos CSV

In [16]:
for key, value in dataframes_dict_551.items():
    file_name = f"Frames_551/frame_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

for key, value in dataframes_dict_605.items():
    file_name = f"Frames_605/frame_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Frames_551/frame_0.csv criado com sucesso.
Arquivo Frames_551/frame_1.csv criado com sucesso.
Arquivo Frames_551/frame_2.csv criado com sucesso.
Arquivo Frames_551/frame_3.csv criado com sucesso.
Arquivo Frames_551/frame_4.csv criado com sucesso.
Arquivo Frames_551/frame_5.csv criado com sucesso.
Arquivo Frames_551/frame_6.csv criado com sucesso.
Arquivo Frames_551/frame_7.csv criado com sucesso.
Arquivo Frames_551/frame_8.csv criado com sucesso.
Arquivo Frames_605/frame_0.csv criado com sucesso.
Arquivo Frames_605/frame_1.csv criado com sucesso.
Arquivo Frames_605/frame_2.csv criado com sucesso.
Arquivo Frames_605/frame_3.csv criado com sucesso.


#### Checa se todos os valores de profundidade presentes no primeiro frame existem nos demais

In [105]:
dataframes_dict_551[0]['TDEP'][0]

720.852

In [104]:
v1 = set(dataframes_dict_551[0]['TDEP'])
v2 = set(dataframes_dict_551[3]['TDEP'])

v1_round = {round(valor, 2) for valor in v1}
v2_round = {round(valor, 2) for valor in v2}

print(v1_round.issubset(v1_round))


True


In [112]:
def checa_TDEP(dataframes):
    tdep_primeiro_frame = dataframes[0]['TDEP']
  
    # Iterar sobre o dicionário de DataFrames
    for key, value in dataframes.items():
        if key == 0:  # Ignorar o primeiro DataFrame
            continue
        
        tdep_outro_frame = set(dataframes[key]['TDEP'])
        
        for valor_tdep in tdep_primeiro_frame:
            if valor_tdep in tdep_outro_frame:
                print(f'{valor_tdep} - Ok')
            else:
                print(f'{valor_tdep} - faltando')
    

In [114]:
checa_TDEP(dataframes_dict_551)

720.8519897460938 - Ok
720.6995849609375 - Ok
720.5471801757812 - Ok
720.394775390625 - Ok
720.2423706054688 - Ok
720.0899658203125 - Ok
719.9375610351562 - Ok
719.7852172851562 - Ok
719.6328125 - Ok
719.4804077148438 - Ok
719.3280029296875 - Ok
719.1755981445312 - Ok
719.023193359375 - Ok
718.8707885742188 - Ok
718.7183837890625 - Ok
718.5659790039062 - Ok
718.41357421875 - Ok
718.2611694335938 - Ok
718.1087646484375 - Ok
717.9564208984375 - Ok
717.8040161132812 - Ok
717.651611328125 - Ok
717.4992065429688 - Ok
717.3468017578125 - Ok
717.1943969726562 - Ok
717.0419921875 - Ok
716.8895874023438 - Ok
716.7371826171875 - Ok
716.5847778320312 - Ok
716.432373046875 - Ok
716.2799682617188 - Ok
716.1275634765625 - Ok
715.9752197265625 - Ok
715.8228149414062 - Ok
715.67041015625 - Ok
715.5180053710938 - Ok
715.3656005859375 - Ok
715.2131958007812 - Ok
715.060791015625 - Ok
714.9083862304688 - Ok
714.7559814453125 - Ok
714.6035766601562 - Ok
714.451171875 - Ok
714.2987670898438 - Ok
714.146362

#### Remove valores de profundidade que não estão presentes no primeiro frame

In [17]:
for key, value in dataframes_dict_551.items():
    dataframes_dict_551[key] = value[value["TDEP"].isin(dataframes_dict_551[0]["TDEP"])]

for key, value in dataframes_dict_605.items():
    dataframes_dict_605[key] = value[value["TDEP"].isin(dataframes_dict_605[0]["TDEP"])]

for key, value in dataframes_dict_1_POI_1_SE.items():
    dataframes_dict_1_POI_1_SE[key] = value[value["TDEP"].isin(dataframes_dict_1_POI_1_SE[0]["TDEP"])]

for key, value in dataframes_dict_696.items():
    dataframes_dict_696[key] = value[value["TDEP"].isin(dataframes_dict_696[0]["TDEP"])]

for key, value in dataframes_dict_1_DP_2_SE.items():
    dataframes_dict_1_DP_2_SE[key] = value[value["TDEP"].isin(dataframes_dict_1_DP_2_SE[0]["TDEP"])]

#### Junta os 9 dataframes do poço 551 em um dataframe unificado

#### Junta os 4 dataframes do poço 605 em um dataframe unificado

In [18]:
dataframe_unificado_551 = unifica_dataframes(dataframes_dict_551)
dataframe_unificado_605 = unifica_dataframes(dataframes_dict_605)
dataframe_unificado_1_POI_1_SE = unifica_dataframes(dataframes_dict_1_POI_1_SE)
dataframe_unificado_696 = unifica_dataframes(dataframes_dict_696)
dataframe_unificado_1_DP_2_SE = unifica_dataframes(dataframes_dict_1_DP_2_SE)

In [19]:
dataframe_unificado_1_DP_2_SE

,TDEP,BS,RT,GR,NPHI,DCAL,DT,HCAL,AHT90,AHF90,HDRA,PEFZ,RHOZ
0,738.835205,8.5,6.018027,72.921120,0.167461,-4.858955,84.478477,3.641045,0.533795,0.532353,-999.250000,-999.25,-999.250000
1,738.682800,8.5,0.533716,72.921120,0.167461,-4.858955,84.478477,3.641045,0.533716,0.532262,-999.250000,-999.25,-999.250000
2,738.530396,8.5,0.533642,72.921120,0.167461,-4.858955,84.478477,3.641045,0.533642,0.532173,-999.250000,-999.25,-999.250000
3,738.377991,8.5,0.533570,72.921120,0.167461,-4.858955,84.478477,3.641045,0.533570,0.532085,-999.250000,-999.25,-999.250000
4,738.225586,8.5,0.533496,72.921120,0.167461,-4.858955,84.478477,3.641045,0.533496,0.531994,-999.250000,-999.25,-999.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4681,25.450800,8.5,13.695553,82.506660,0.643228,-4.880896,56.886055,3.619104,13.695553,13.236780,0.164936,10.00,2.180906
4682,25.298399,8.5,13.695553,86.358650,0.682383,-4.880896,56.886055,3.619104,13.695553,13.236780,0.164936,10.00,2.180906
4683,25.146000,8.5,13.695553,91.280937,0.741800,-4.880896,56.886055,3.619104,13.695553,13.236780,0.164936,10.00,2.180906
4684,24.993599,8.5,13.695553,83.545921,0.804134,-4.880896,56.886055,3.619104,13.695553,13.236780,0.164936,10.00,2.180906


#### Mantém apenas uma curva de resistividade profunda

In [20]:
# manter RT
dataframe_unificado_filtrado_551 = remove_colunas(dataframe_unificado_551, ['AHT90', 'AHF90'])
dataframe_unificado_filtrado_1_POI_1_SE = remove_colunas(dataframe_unificado_1_POI_1_SE, ['AHT90', 'AHF90'])
dataframe_unificado_filtrado_696 = remove_colunas(dataframe_unificado_551, ['AHT90', 'AHF90'])
dataframe_unificado_filtrado_1_DP_2_SE = remove_colunas(dataframe_unificado_1_DP_2_SE, ['AHT90', 'AHF90'])

In [21]:
dataframe_unificado_551

,TDEP,BS,RT,GR,NPHI,DCAL,DT,HCAL,AHT90,AHF90,HDRA,PEFZ,RHOZ
0,720.851990,8.5,16.035276,129.733124,0.691645,-5.699512,99.898071,2.800488,1.603695,1.639400,-999.250000,-999.25,-999.250000
1,720.699585,8.5,1.603191,129.733124,0.691645,-5.699512,99.898071,2.800488,1.603191,1.639136,-999.250000,-999.25,-999.250000
2,720.547180,8.5,1.601933,129.733124,0.691645,-5.699512,99.898071,2.800488,1.601933,1.638554,-999.250000,-999.25,-999.250000
3,720.394775,8.5,1.599770,129.733124,0.691645,-5.699512,99.898071,2.800488,1.599770,1.637244,-999.250000,-999.25,-999.250000
4,720.242371,8.5,1.594869,129.733124,0.691645,-5.699512,100.353745,2.800488,1.594869,1.634789,-999.250000,-999.25,-999.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,27.279600,8.5,15.466356,19.685118,0.684838,-5.745168,78.800926,2.754833,15.466356,15.257225,0.026757,10.00,2.283773
4552,27.127199,8.5,15.466356,20.519228,0.664183,-5.745168,78.800926,2.754833,15.466356,15.257225,0.026757,10.00,2.283773
4553,26.974800,8.5,15.466356,25.015450,0.690279,-5.745168,78.800926,2.754833,15.466356,15.257225,0.026757,10.00,2.283773
4554,26.822399,8.5,15.466356,24.739817,0.690279,-5.745168,78.800926,2.754833,15.466356,15.257225,0.026757,10.00,2.283773


In [22]:
dataframe_unificado_605

,TDEP,BS,GR,NPHI,DT,HCAL,AHF90,HDRA,PEFZ,RHOZ
0,671.169617,8.5,47.261490,0.263958,82.793983,3.887136,0.812778,-999.250000,-999.250000,-999.25000
1,671.017212,8.5,47.261490,0.263958,82.793983,3.887136,0.812771,-999.250000,-999.250000,-999.25000
2,670.864807,8.5,47.261490,0.263958,82.793983,3.887136,0.811494,-999.250000,-999.250000,-999.25000
3,670.712402,8.5,47.261490,0.263958,82.793983,3.887136,0.810167,-999.250000,-999.250000,-999.25000
4,670.559998,8.5,47.261490,0.263958,82.793983,3.887136,0.809051,-999.250000,-999.250000,-999.25000
...,...,...,...,...,...,...,...,...,...,...
4235,25.755600,8.5,59.688400,0.463366,60.000000,3.812884,7.044221,0.178654,9.243673,2.14924
4236,25.603199,8.5,59.507236,0.459518,60.000000,3.812884,7.044221,0.178654,9.243673,2.14924
4237,25.450800,8.5,59.372772,0.460849,60.000000,3.802276,7.044221,0.178654,9.243673,2.14924
4238,25.298399,8.5,60.822781,0.484079,60.000000,3.823491,7.044221,0.178654,9.243673,2.14924


In [23]:
dataframe_unificado_filtrado_696

,TDEP,BS,RT,GR,NPHI,DCAL,DT,HCAL,HDRA,PEFZ,RHOZ
0,720.851990,8.5,16.035276,129.733124,0.691645,-5.699512,99.898071,2.800488,-999.250000,-999.25,-999.250000
1,720.699585,8.5,1.603191,129.733124,0.691645,-5.699512,99.898071,2.800488,-999.250000,-999.25,-999.250000
2,720.547180,8.5,1.601933,129.733124,0.691645,-5.699512,99.898071,2.800488,-999.250000,-999.25,-999.250000
3,720.394775,8.5,1.599770,129.733124,0.691645,-5.699512,99.898071,2.800488,-999.250000,-999.25,-999.250000
4,720.242371,8.5,1.594869,129.733124,0.691645,-5.699512,100.353745,2.800488,-999.250000,-999.25,-999.250000
...,...,...,...,...,...,...,...,...,...,...,...
4551,27.279600,8.5,15.466356,19.685118,0.684838,-5.745168,78.800926,2.754833,0.026757,10.00,2.283773
4552,27.127199,8.5,15.466356,20.519228,0.664183,-5.745168,78.800926,2.754833,0.026757,10.00,2.283773
4553,26.974800,8.5,15.466356,25.015450,0.690279,-5.745168,78.800926,2.754833,0.026757,10.00,2.283773
4554,26.822399,8.5,15.466356,24.739817,0.690279,-5.745168,78.800926,2.754833,0.026757,10.00,2.283773


## Cria dataframes para os demais poços

In [24]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():

    # Armazenando as curvas que serão utilizadas em uma lista
    curvas_utilizadas = [
        channel.name                                    # Os elementos da lista serão os nomes das curvas
        for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
        if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
    ]
    conjunto_aux = set(curvas_utilizadas)
    curvas_utilizadas_sem_duplicados = list(conjunto_aux)
    
    #frames = [np.atleast_1d(frame) for frame in poco.frames]
    #curvas = np.concatenate(frames, axis=0)
    try:
        dataframe = pd.DataFrame()
        
        for frame in poco.frames:

            curvas = frame.curves()

            dataframe = pd.concat([dataframe, pd.DataFrame(curvas[curvas_utilizadas_sem_duplicados])])

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = dataframe
    except:
        pass


In [25]:
dlis_df_dict['1-BRSA-659-SE'] = remove_colunas(dlis_df_dict['1-BRSA-659-SE'], ['PEU'])
dlis_df_dict['1-FSJQ-1-SE'] = remove_colunas(dlis_df_dict['1-FSJQ-1-SE'], ['PEU'])

In [26]:
dlis_df_dict['1-BRSA-605-SE'] = dataframe_unificado_605
dlis_df_dict['1-FSG-1-SE'] = dataframe_unificado_filtrado_551
dlis_df_dict['1-POI-1-SE'] = dataframe_unificado_filtrado_1_POI_1_SE
dlis_df_dict['1-BRSA-696-SE'] = dataframe_unificado_filtrado_696
dlis_df_dict['1-DP-2-SE'] = dataframe_unificado_filtrado_1_DP_2_SE
dlis_df_dict.keys()

dict_keys(['1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-659-SE', '1CSC 0001  SE', '1FBV 0001  SE', '1PRP 0001  SE', '1-BRSA-605-SE', '1-FSG-1-SE', '1-POI-1-SE', '1-BRSA-696-SE', '1-DP-2-SE'])

## Transforma os valores -999.25 em nulos

In [27]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [28]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD',	'LL7',	'RLLD',	 'RLL7', 'HDRS', 'HLLD', 'ILD',	'RILD',	'IEL',	'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90',	'AHF90', 'AFH90'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')
aplica_mnemonico(dlis_df_dict, ['DTC', 'DT'], 'DT')
aplica_mnemonico(dlis_df_dict, ['HCAL', 'CAL', 'CALI'], 'CAL')
aplica_mnemonico(dlis_df_dict, ['DCAL', 'DCALI'], 'DCAL')
aplica_mnemonico(dlis_df_dict, ['DRHO', 'HDRA'], 'DRHO')
aplica_mnemonico(dlis_df_dict, ['PE', 'PEFZ', 'PEU'], 'PE')

## Adiciona coluna DCAL

In [29]:
add_DCAL(dlis_df_dict)

In [30]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

1-FSJQ-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-595-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-659-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1CSC 0001  SE: ['CAL', 'DCAL', 'DT', 'GR', 'RESD']
1FBV 0001  SE: ['DCAL', 'DUMM', 'FRAMENO', 'INDEX507']
1PRP 0001  SE: ['DCAL', 'GR']
1-BRSA-605-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-FSG-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-POI-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-696-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-DP-2-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Preenche os poços com curvas faltando (não é mais necessário)

In [31]:
'''
# Se um dos poços não tiver uma dessas curvas, adicionamos a coluna da curva e mantemos os valores como None
curvas_obrigatorias = ['TDEP', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

# Percorre todos os poços
for poco in dlis_df_dict.values():
    # Percorre todas as curvas obrigatórias
    for curva in curvas_obrigatorias:
        # Se o poço não tiver a curva
        if curva not in poco.columns:
            # Adiciona a coluna e os valores dela = None
            poco[curva] = None
'''

## Temos as seguintes curvas 

In [32]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

1-FSJQ-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-595-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-659-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1CSC 0001  SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1FBV 0001  SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'DUMM', 'FRAMENO', 'GR', 'INDEX507', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1PRP 0001  SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-605-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-FSG-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-POI-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-696-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-DP-2-SE: ['BS', 'CAL', 'D

## Remove valores DRHO e DCAL indesejados (Só depois)

In [30]:
#limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
#limita_curva(dlis_df_dict, "DCAL", -2, 2)

## Inverte a ordem das linhas dos dataframes

In [62]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Ordena as colunas dos dataframes

In [63]:
# Simon e Vandelli já definiram (maio/24)
ordem_desejada = ['TDEP', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

for key in dlis_df_dict.keys():
    try:
        dlis_df_dict[key] = dlis_df_dict[key].reindex(columns=ordem_desejada)
    except:
        pass

## Salva os dados dos dataframes em arquivos CSV

In [64]:
for key, value in dlis_df_dict.items():
    file_name = f"Pocos-pre-processados/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Pocos-pre-processados/poco_1-FSJQ-1-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-595-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-659-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1CSC 0001  SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1FBV 0001  SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1PRP 0001  SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-605-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-FSG-1-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-POI-1-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-696-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-DP-2-SE.csv criado com sucesso.
